In [ ]:
import os
from time import sleep

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils import rows_from_range

from collections import defaultdict
from tqdm.notebook import tqdm
from dataclasses import dataclass, field
from rapidfuzz import fuzz

dow = r'E:\sirne\Downloads'

In [ ]:
browser = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
browser.get('https://audio-english.ru/frequencydict/s_1_po_500/')
words = []
prev_link = ""

while browser.current_url != prev_link:
    prev_link = browser.current_url

    rows = browser.find_elements(By.TAG_NAME, "tr")[1:]
    for r in rows:
        td = r.find_elements(By.TAG_NAME, "td")
        words.append([td[c].text for c in [2, 4, 7]])

    nxt = browser.find_element(
        By.CSS_SELECTOR, "div.section.les-page-nav.clearfix")
    nxt = nxt.find_elements(By.TAG_NAME, "li")[-1]
    nxt.click()

In [ ]:
lines = ["\t".join([f'"{t.strip()}"' if '\n' in t else t.strip()
                   for t in line]) for line in words]
text = "\n".join(lines)

with open(f'{dow}\\words.txt', 'w', encoding='utf8') as f:
    f.write(text)

In [ ]:
result = Workbook()
resSheet = result.active
data_size = 0

wb = load_workbook(f'{dow}\\2.xlsx')

for sheet in wb:
    for row in range(1, sheet.max_row+1):
        for cell in range(1, sheet.max_column+1):
            resSheet.cell(
                row=data_size+row, column=cell).value = sheet.cell(row=row, column=cell).value
    data_size += sheet.max_row

result.save(f'{dow}\\words.xlsx')

In [ ]:
browser.get('https://www.englishclub.com/ref/Collocations/')

In [ ]:
pages = [el.find_element(By.TAG_NAME, 'a').get_attribute('href') for el in browser.find_element(
    By.CSS_SELECTOR, "#wide_cats").find_elements(By.CSS_SELECTOR, "dt.ec-big1")[:-1]]

In [ ]:
print(ph, de)

In [ ]:
words = []

for page in pages:
    browser.get(page)
    for el in browser.find_elements(By.CSS_SELECTOR, "div.linklisting"):
        words.append([
            el.find_element(By.CSS_SELECTOR, n).text.strip()
            for n in [".linktitle", ".linkdescription"
                      ]])

In [ ]:
wb = load_workbook(f'{dow}\\wordsz.xlsx', read_only=True)

In [ ]:
@dataclass
class Word:
    word: str = ""
    translates: set = field(default_factory=set)
    transcription: str = ""
    comments: set = field(default_factory=set)
    iscapital: bool = False

    def __iadd__(self, row):
        self.word = min(row[0], self.word) if self.word else row[0]
        self.iscapital = self.word[0].istitle() or self.iscapital

        trans = row[1]
        if trans:
            for i in list(self.translates):
                tl, il = [j.lower() for j in [trans, i]]
                if tl in il:
                    break
                if il in tl:
                    self.translates.remove(i)
            else:
                self.translates.add(trans)

        if row[2]:
            r = row[2]
            if fuzz.QRatio(r, self.transcription) > 25 or not self.transcription:
                self.transcription = max(r, self.transcription)
            else:
                self.comments.add(r)

        if row[3]:
            self.comments.add(row[3])

        return self

In [ ]:
def read_sheet(sheet, DataClass):
    coll = defaultdict(lambda: DataClass())

    with tqdm(total=sheet.max_row) as pbar:
        for row in sheet.rows:
            values = [str(cell.value).replace('\xa0', ' ').strip()
                      if cell.value else cell.value
                      for cell in row]
            if values[0]:
                coll[values[0].lower()] += values
            pbar.update(1)
    return coll

In [ ]:
coll = read_sheet(wb[wb.sheetnames[1]], Word)
print(len(coll))

In [ ]:
with open(f'{dow}\\words.txt', 'w', encoding='utf8') as f:
    for c in coll:
        if len(coll[c].translates) > 2:
            l = list(coll[c].translates)
            for i in range(len(l)-1):
                for j in range(i+1, len(l)):
                    r = fuzz.QRatio(l[i], l[j])
                    if 0 < r < 100:
                        f.write(
                            f'{r:.2f}\t"{l[i]}"\t"{l[j]}"\t"{coll[c].transcription}"\n')
#                         print(f"{r:.2f}\t'{l[i]}'\t'{l[j]}'")

In [ ]:
with open(f'{dow}\\words_uniq.txt', 'w', encoding='utf8') as f, open(f'{dow}\\words.txt', 'w', encoding='utf8') as u, open(f'{dow}\\words_cap.txt', 'w', encoding='utf8') as cap:
    for i in coll.values():
        comment = "\n".join(i.comments)
        trans = "\n".join(i.translates)
        if not i.iscapital:
            f.write(f'"{i.word}"\t"{trans}"\t"{i.transcription}"\t"{comment}"\n')
            for tr in i.translates:
                u.write(
                    f'"{i.word}"\t"{tr}"\t"{i.transcription}"\t"{comment}"\n')
        else:
            cap.write(
                f'"{i.word}"\t"{trans}"\t"{i.transcription}"\t"{comment}"\n')

In [ ]:
@dataclass
class Phrase:
    phrase: str = ""
    translates: set = field(default_factory=set)

    def __iadd__(self, row):
        self.phrase = min(row[0], self.phrase) if self.phrase else row[0]

        trans = row[1]
        if trans:
            for i in list(self.translates):
                tl, il = [j.lower() for j in [trans, i]]
                if tl in il:
                    break
                if il in tl:
                    self.translates.remove(i)
            else:
                self.translates.add(trans)
        return self

In [ ]:
phrases = read_sheet(wb[wb.sheetnames[2]], Phrase)
print(len(phrases))

In [ ]:
p = phrases
for i in p:
    if len(p[i].translates) > 1:
        print(i, len(p[i].translates), p[i].translates)

In [ ]:
with open(f'{dow}\\phrases.txt', 'w', encoding='utf8') as f:
    for i in phrases.values():
        trans = "\n".join(i.translates)
        f.write(f'"{i.phrase}"\t"{trans}"\n')

In [ ]:
wb.close()